In [1]:
# Check redshift/mass bins
from mceh import utility as ut # v3.0.0-alpha.2
import numpy as np
import matplotlib.pyplot as plt
import mceh.utility as ut
import mceh.find_luminosity as fl
import astropy.units as u

Call manual() to get the manual.


In [12]:
efeds, hsc = ut.init('efeds', 'hsc')

In [2]:
result = ut.pickle_load('result/bkg20241108.pickle')

In [4]:
result.keys()

dict_keys(['ra', 'dec', 'unmasked_fraction', 'hsc_index', 'r', 'hsc_version'])

In [13]:
bins = np.arange(10, 30.1, 0.2)

In [ ]:
filter_name = ['gmag_cmodel', 'rmag_cmodel', 'imag_cmodel', 'zmag_cmodel', 
               'ymag_cmodel']
rd_num = len(result['ra'])
all_rd_raw_lf = {}
all_rd_corrected_lf = {}
for fn in filter_name:
    filter_raw_lf = []
    filter_corrected_lf = []
    for i in range(rd_num):
        index, umfrac = result['hsc_index'][i], result['unmasked_fraction'][i]
        original_lf = np.histogram(hsc[fn][index], bins)[0]
        corrected_lf = original_lf / umfrac.value
        filter_raw_lf.append(original_lf)
        filter_corrected_lf.append(corrected_lf)
    all_rd_raw_lf[fn] = filter_raw_lf
    all_rd_corrected_lf[fn] = filter_corrected_lf

In [ ]:
bins[20] # Have values from this on

13.999999999999986

In [60]:
mean = {}
std = {}
for fn in filter_name:
    mean[fn] = np.mean(all_rd_corrected_lf[fn], axis = 0)
    std[fn] = np.std(all_rd_corrected_lf[fn], axis = 0)

In [62]:
def area(r):
    return 2 * np.pi * (1 - np.cos(r)) * u.rad**2

In [64]:
mean

{'gmag_cmodel': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.13090947e-04,
        4.80716180e-04, 0.00000000e+00, 2.01257395e-03, 8.20360401e-04,
        2.59648345e-03, 3.71870343e-03, 8.75658896e-03, 1.32010188e-02,
        2.12035497e-02, 3.70249316e-02, 4.61805396e-02, 6.09604877e-02,
        6.79443964e-02, 1.00804066e-01, 1.39271633e-01, 1.66413619e-01,
        2.31847692e-01, 2.70547885e-01, 3.50087039e-01, 4.24957588e-01,
        5.27290097e-01, 6.64146848e-01, 7.76436975e-01, 9.19272294e-01,
        1.13257282e+00, 1.39660251e+00, 1.6976643

In [69]:
a = (1 * u.arcmin) / (2 * u.arcmin)
b = np.array([a])
type(b[0])

numpy.float64

In [73]:
result2 = {}
result2['filter'] = filter_name
result2['area'] = area(result['r'])
result2['unmasked_fraction'] = np.array(result['unmasked_fraction'])
result2['all_raw_lf'] = np.array([all_rd_raw_lf[fn] for fn in filter_name])
result2['all_corrected_lf'] = np.array([all_rd_corrected_lf[fn] for fn in filter_name])
result2['mean_lf'] = np.array([mean[fn] for fn in filter_name])
result2['std_lf'] = np.array([std[fn] for fn in filter_name])
result2['mean_lf_d'] = result2['mean_lf'] / (result2['area'].to(u.arcmin**2))
result2['std_lf_d'] = result2['std_lf'] / result2['area'].to(u.arcmin**2)

In [80]:
ut.pickle_dump(result2, 'bkg_lf20241111.pickle')